In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
mep = pd.read_csv('retweets.csv')

In [3]:
mep.shape

(26133, 21)

In [4]:
mep.columns

Index(['origUserId', 'origUserScreenName', 'origMepId', 'origMepName',
       'origMepGroupId', 'origMepGroupShort', 'origMepCountryId',
       'origMepCountryShort', 'retweetUserId', 'retweetUserScreenName',
       'retweetMepId', 'retweetMepName', 'retweetMepGroupId',
       'retweetMepGroupShort', 'retweetMepCountryId', 'retweetMepCountryShort',
       'origCreatedAt', 'origTweetId', 'retweetCreatedAt', 'retweetTweetId',
       'lang'],
      dtype='object')

Delete the records with not English languages

In [5]:
mep.lang

0         el
1         el
2         fr
3         en
4         fr
        ... 
26128     en
26129    und
26130     fr
26131     fr
26132     es
Name: lang, Length: 26133, dtype: object

In [6]:
mep = mep.loc[mep.lang == 'en']

In [7]:
import tweepy
auth = tweepy.OAuthHandler('hsaHW0P7kFtP7yiOTWV6E5Cmz', 'A1fgwDsjBQ48dYM8xHqq1fafs8sJpOAj9m9QFkEmqBjorpEk4X')

In [8]:
mep

,origUserId,origUserScreenName,origMepId,origMepName,origMepGroupId,origMepGroupShort,origMepCountryId,origMepCountryShort,retweetUserId,retweetUserScreenName,...,retweetMepName,retweetMepGroupId,retweetMepGroupShort,retweetMepCountryId,retweetMepCountryShort,origCreatedAt,origTweetId,retweetCreatedAt,retweetTweetId,lang
3,19017675,Nigel_Farage,4525,Nigel FARAGE,6,EFDD,27,GBR,121171051,MargotLJParker,...,Margot PARKER,6,EFDD,27,GBR,Wed Jul 04 11:05:03 +0000 2012,220473289259233285,Sun Dec 07 21:46:00 +0000 2014,541710240014942209,en
5,17675072,MartinSchulz,1911,Martin SCHULZ,1,S&D,10,DEU,1668992125,NathanGillMEP,...,Nathan GILL,6,EFDD,27,GBR,Fri Jul 05 08:00:23 +0000 2013,353060776707235841,Thu Jun 18 08:52:51 +0000 2015,611456527409524736,en
11,17675072,MartinSchulz,1911,Martin SCHULZ,1,S&D,10,DEU,21648649,maritaulvskog,...,Marita ULVSKOG,1,S&D,26,SWE,Sat May 17 12:47:17 +0000 2014,467647542378123264,Wed Sep 16 09:15:06 +0000 2015,644077036022165504,en
12,744743599,Tim_Aker,99650,Tim AKER,6,EFDD,27,GBR,121171051,MargotLJParker,...,Margot PARKER,6,EFDD,27,GBR,Wed May 28 16:47:44 +0000 2014,471694322472353793,Thu Dec 04 22:21:10 +0000 2014,540631925145493504,en
13,744743599,Tim_Aker,99650,Tim AKER,6,EFDD,27,GBR,19017675,Nigel_Farage,...,Nigel FARAGE,6,EFDD,27,GBR,Wed May 28 16:47:44 +0000 2014,471694322472353793,Sun Mar 29 18:53:32 +0000 2015,582254277122473984,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26121,202610289,akhanmep,124962,Afzal KHAN,1,S&D,27,GBR,829060856,TheresaMEP,...,Theresa GRIFFIN,1,S&D,27,GBR,Sun Jan 24 14:39:12 +0000 2016,691269020667002880,Sun Jan 24 14:40:33 +0000 2016,691269361059893249,en
26122,202610289,akhanmep,124962,Afzal KHAN,1,S&D,27,GBR,1354570123,julie4nw,...,Julie WARD,1,S&D,27,GBR,Sun Jan 24 14:39:12 +0000 2016,691269020667002880,Sun Jan 24 14:54:13 +0000 2016,691272799604883457,en
26126,202610289,akhanmep,124962,Afzal KHAN,1,S&D,27,GBR,829060856,TheresaMEP,...,Theresa GRIFFIN,1,S&D,27,GBR,Sun Jan 24 17:19:25 +0000 2016,691309340691660800,Sun Jan 24 17:24:28 +0000 2016,691310613272883200,en
26127,202610289,akhanmep,124962,Afzal KHAN,1,S&D,27,GBR,1354570123,julie4nw,...,Julie WARD,1,S&D,27,GBR,Sun Jan 24 17:19:25 +0000 2016,691309340691660800,Sun Jan 24 18:37:38 +0000 2016,691329024556163072,en


In [102]:
origTweetList = mep.origTweetId.tolist()

In [10]:
try:
    redirect_url = auth.get_authorization_url()
except tweepy.TweepError:
    print('Error! Failed to get request token.')

In [11]:
# api = tweepy.API(auth)
# origText = []
# for tweetid in origTweetList:
#     try:
#         tweet = api.get_status(tweetid)
#         origText.append(tweet.text)
#         print(tweet.text)
#     except:
#         origText.append('')
#         print(' ')

https://stackoverflow.com/questions/44581647/retrieving-a-list-of-tweets-using-tweet-id-in-tweepy

In [12]:
import tweepy

import time

t0 = time.time()

def lookup_tweets(tweet_IDs, api):
    full_tweets = []
    tweet_count = len(tweet_IDs)
    try:
        for i in range((tweet_count // 100) + 1):
            # Catch the last group if it is less than 100 tweets
            end_loc = min((i + 1) * 100, tweet_count)
            full_tweets.extend(
                api.statuses_lookup(id_=tweet_IDs[i * 100:end_loc])
            )  
        return full_tweets
    except tweepy.TweepError:
        print('Something went wrong, quitting...')

auth = tweepy.OAuthHandler('hsaHW0P7kFtP7yiOTWV6E5Cmz', 'A1fgwDsjBQ48dYM8xHqq1fafs8sJpOAj9m9QFkEmqBjorpEk4X')

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# do whatever it is to get por.TweetID - the list of all IDs to look up

results = lookup_tweets(origTweetList, api)

final_dic = {}

t1 = time.time()
total = t1-t0
print(total)

280.1609683036804


In [25]:
for tweet in results:
    if tweet:
        final_dic[tweet.id] = tweet.text

In [26]:
import pprint
# pprint.pprint(final_dic)
len(final_dic)

10126

In [27]:
tweet_text = pd.DataFrame.from_dict(final_dic, orient='index', columns=['tweetText'])

## Cross Validate that the number of rows is **10126**

### Something to investigate: Do we have duplicate tweets on the dataset?
### mep rows: 13796
### mep.origTweetId.unique(): 10834

In [95]:
len(mep.origTweetId.unique())

10834

In [136]:
tweet_text

,tweetText
519050739642286080,Lord Hill's written answers to EP are now on m...
519139901359783936,Our @SebDance asks @ABratusek about global cli...
518024241024073728,I write for @LabourList Tory threat to Court o...
519109106272317440,"“@Ashleyfoxmep: and named her party ""the Allia..."
518721908951769088,Good luck to @TheProgressives in Bulgaria toda...
...,...
690861330555887616,Memo to Tories for Reform in EU: Never mind th...
690514426991120384,#EUreferendum is going to be about working peo...
690096989292367872,Extremely rich corporate folk who do extremely...
690207644838133760,I have called upon the Indian government to re...


In [119]:
mepExt = pd.merge(mep, tweet_text, left_on='origTweetId', right_index=True, how='inner')

In [120]:
group = mepExt.groupby('origMepGroupShort').count()
groupFiltered = group.loc[group.origUserId > 50]
mepExt = mepExt.loc[mepExt.origMepGroupShort.isin(list(groupFiltered.index))]

In [121]:
mepExt

,origUserId,origUserScreenName,origMepId,origMepName,origMepGroupId,origMepGroupShort,origMepCountryId,origMepCountryShort,retweetUserId,retweetUserScreenName,...,retweetMepGroupId,retweetMepGroupShort,retweetMepCountryId,retweetMepCountryShort,origCreatedAt,origTweetId,retweetCreatedAt,retweetTweetId,lang,tweetText
3,19017675,Nigel_Farage,4525,Nigel FARAGE,6,EFDD,27,GBR,121171051,MargotLJParker,...,6,EFDD,27,GBR,Wed Jul 04 11:05:03 +0000 2012,220473289259233285,Sun Dec 07 21:46:00 +0000 2014,541710240014942209,en,Euro Parliament votes overwhelmingly to reject...
5,17675072,MartinSchulz,1911,Martin SCHULZ,1,S&D,10,DEU,1668992125,NathanGillMEP,...,6,EFDD,27,GBR,Fri Jul 05 08:00:23 +0000 2013,353060776707235841,Thu Jun 18 08:52:51 +0000 2015,611456527409524736,en,"US have one currency, one Central Bank and one..."
11,17675072,MartinSchulz,1911,Martin SCHULZ,1,S&D,10,DEU,21648649,maritaulvskog,...,1,S&D,26,SWE,Sat May 17 12:47:17 +0000 2014,467647542378123264,Wed Sep 16 09:15:06 +0000 2015,644077036022165504,en,More jobs for young people is my top priority ...
12,744743599,Tim_Aker,99650,Tim AKER,6,EFDD,27,GBR,121171051,MargotLJParker,...,6,EFDD,27,GBR,Wed May 28 16:47:44 +0000 2014,471694322472353793,Thu Dec 04 22:21:10 +0000 2014,540631925145493504,en,"Vote #Labour, get Tory. Vote Tory, get Labour...."
13,744743599,Tim_Aker,99650,Tim AKER,6,EFDD,27,GBR,19017675,Nigel_Farage,...,6,EFDD,27,GBR,Wed May 28 16:47:44 +0000 2014,471694322472353793,Sun Mar 29 18:53:32 +0000 2015,582254277122473984,en,"Vote #Labour, get Tory. Vote Tory, get Labour...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26114,1485429175,JSaryuszWolski,28297,Jacek SARYUSZ-WOLSKI,4,EPP,20,POL,539156512,ZdzKrasnodebski,...,5,ECR,20,POL,Sun Jan 24 13:17:25 +0000 2016,691248438504210432,Mon Jan 25 08:21:14 +0000 2016,691536290198745088,en,Drastic drop in positive EU perception in Pola...
26117,829060856,TheresaMEP,124961,Theresa GRIFFIN,1,S&D,27,GBR,1354570123,julie4nw,...,1,S&D,27,GBR,Sun Jan 24 14:04:13 +0000 2016,691260217288454144,Sun Jan 24 14:54:31 +0000 2016,691272874393485314,en,I'm with @UKYP North West now in Knowsley - gr...
26118,829060856,TheresaMEP,124961,Theresa GRIFFIN,1,S&D,27,GBR,202610289,akhanmep,...,1,S&D,27,GBR,Sun Jan 24 14:12:29 +0000 2016,691262296396599296,Sun Jan 24 14:21:13 +0000 2016,691264493725335552,en,Listening to some fab ideas from young people ...
26119,829060856,TheresaMEP,124961,Theresa GRIFFIN,1,S&D,27,GBR,1354570123,julie4nw,...,1,S&D,27,GBR,Sun Jan 24 14:12:29 +0000 2016,691262296396599296,Sun Jan 24 14:54:26 +0000 2016,691272856739659776,en,Listening to some fab ideas from young people ...


## Finish the clearing part -- Exporting the CSV so I don't need to run the proccess over and over again

In [128]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [125]:
export = mepExt.to_csv(r'C:\Users\Ion\Documents\GitHub\Clustering-and-Classification-of-MEPs-Tweets\fullRetweets.csv',  index = None, header=True)

In [126]:
fullRetweets = pd.read_csv('FullRetweets.csv')

## Remember to do something with word clouds

In [161]:
import wordcloud
import matplotlib
import PIL

In [182]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn import metrics
from nltk.corpus import stopwords
import nltk
import sklearn as sk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ion\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [171]:
corpus = list(tweet_text.tweetText)
# corpus = [x.lower() for x in corpus]

In [175]:
my_stop_words = set(stopwords.words('english')) 
vectorizer = CountVectorizer(lowercase=True, min_df=10, max_df=0.5, stop_words =set(my_stop_words))
X = vectorizer.fit_transform(corpus)

In [176]:
print(vectorizer.get_feature_names())

['00', '000', '10', '100', '11', '12', '13', '14', '15', '16', '17', '18', '19', '1st', '20', '2010', '2012', '2014', '2015', '2016', '2017', '2020', '22', '24', '25', '27', '28', '2nd', '30', '3bn', '3rd', '40', '45', '50', '70', '75', '80', 'able', 'abroad', 'absolutely', 'abuse', 'accept', 'accepted', 'access', 'according', 'account', 'achieve', 'across', 'act', 'action', 'actions', 'activists', 'actually', 'address', 'addressing', 'admits', 'adopted', 'advice', 'affairs', 'affected', 'africa', 'african', 'afternoon', 'age', 'agenda', 'ago', 'agree', 'agreed', 'agreement', 'agreements', 'agrees', 'agriculture', 'ahead', 'aid', 'air', 'akhanmep', 'al', 'alan', 'alde', 'aldecongress', 'aldegroup', 'aldeparty', 'allianceecr', 'allow', 'allowed', 'allowing', 'almost', 'alone', 'along', 'alongside', 'already', 'also', 'alternative', 'always', 'amazing', 'ambassador', 'ambition', 'ambitious', 'amendment', 'amendments', 'american', 'among', 'amp', 'anagomesmep', 'analysis', 'andreas_schwab

In [177]:
kmeans = KMeans(n_clusters=2).fit(X)

In [184]:
cluster_labels = kmeans.fit_predict(X)

In [185]:
sk.metrics.silhouette_score(X, labels=cluster_labels)

0.030033335325703223